In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
from pyspark.sql.window import Window 
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.config('spark.driver.memory','4g').getOrCreate()
Filepath = "D:\\LEARNING\\Study_DE\\Data\\Dataset\\log_search\\"
Mappath1 = "D:\\PROJECT\\Data-Engineering-and-Big-Data\\mapthang6.csv"
Mappath2 = "D:\\PROJECT\\Data-Engineering-and-Big-Data\\mapthang7.csv"
outputpath = "D:\\OUTPUTDATA"

In [2]:
def ReadFileAndUnion(Path,FromDate,ToDate):
    schema = StructType([
    StructField("eventID", StringType(), True),
    StructField("datetime", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("keyword", StringType(), True),
    StructField("category", StringType(), True),
    StructField("proxy_isp", StringType(), True),
    StructField("platform", StringType(), True),
    StructField("networkType", StringType(), True),
    StructField("action", StringType(), True),
    StructField("userPlansMap", ArrayType(StringType()), True),
    ])
    data = spark.createDataFrame([],schema = schema)

    listSTR = []
    listINT = list(range(FromDate,ToDate+1))
    for item in listINT:
        listSTR.append(str(item))
    filesname = [Path + fname for fname in listSTR]
    filecount = 1

    for name in filesname:
        df = spark.read.parquet(name)
        data = data.union(df)
        print(f"Done {filecount} day")
        filecount = filecount + 1
    
    return data

In [3]:
data6 = ReadFileAndUnion(Filepath,20220601,20220614)
data7 = ReadFileAndUnion(Filepath,20220701,20220714)

Done 1 day
Done 2 day
Done 3 day
Done 4 day
Done 5 day
Done 6 day
Done 7 day
Done 8 day
Done 9 day
Done 10 day
Done 11 day
Done 12 day
Done 13 day
Done 14 day
Done 1 day
Done 2 day
Done 3 day
Done 4 day
Done 5 day
Done 6 day
Done 7 day
Done 8 day
Done 9 day
Done 10 day
Done 11 day
Done 12 day
Done 13 day
Done 14 day


In [4]:
data6.show()

+--------------------+--------------------+--------+--------------------+--------+---------+--------------------+-----------+------+--------------------+
|             eventID|            datetime| user_id|             keyword|category|proxy_isp|            platform|networkType|action|        userPlansMap|
+--------------------+--------------------+--------+--------------------+--------+---------+--------------------+-----------+------+--------------------+
|61804316-6b89-4cf...|2022-06-01 18:59:...|    NULL|            trữ tình|   enter|     vnpt|   fplay-ottbox-2019|   ethernet|search|                NULL|
|22c35287-9fe1-487...|2022-06-01 18:59:...|44887906|            trữ tình|   enter|     vnpt|   fplay-ottbox-2019|   ethernet|search|                  []|
|f9af5a95-1f72-486...|2022-06-01 18:59:...| 2719170|              bolero|   enter|  viettel|   fplay-ottbox-2019|   ethernet|search|[Kênh Gia Đình:pr...|
|fd53edee-132d-4ac...|2022-06-01 15:00:...|    NULL|amy schumer: trực...|   

In [5]:
data6 = data6.select('user_id','keyword')

In [6]:
data6.show()

+--------+--------------------+
| user_id|             keyword|
+--------+--------------------+
|    NULL|            trữ tình|
|44887906|            trữ tình|
| 2719170|              bolero|
|    NULL|amy schumer: trực...|
| 8830996|cậu mang à sĩ hanako|
|    NULL|      Hoa trong bao |
|41559909|liên minh công lý...|
|92715770|                NULL|
|49026196|    việt nam vs appa|
|    NULL|chuyển sinh thành...|
|41376437|nhất kiến khuynh tâm|
| 1254139|                giác|
|42534799|            nexsport|
|49190631|Tìm kiếm bằng giọ...|
|    NULL|Lương Sơn Bá Chúc...|
|91485275|một mảnh băng tâm...|
|45282582|            sáng tạo|
|41851691|                NULL|
| 2599280|            truc tep|
|41681697|                NULL|
+--------+--------------------+
only showing top 20 rows



In [7]:
data6 = data6.groupBy('user_id','keyword').count()

In [8]:
data6.show()

+--------+--------------------+-----+
| user_id|             keyword|count|
+--------+--------------------+-----+
| 8460879|Đao Kiếm Thần Vực...|    1|
| 3040638|             bóng đá|    1|
|98543977|    tái sinh báo thù|    1|
| 9030455|Trực tiếp U23 Việ...|    1|
|41789116|   tiên kiếm kỳ hiệp|    1|
|91999697|        bóng đá u23 |    1|
| 8221328|           kênh vtv6|    1|
| 0789087|    giao hữu quốc tế|    1|
| 0508207|không thèm yêu đư...|    3|
|42363805|             bóng đá|    1|
|49603005|truc tiep bong đá...|    1|
|42797857|hãy gọi tôi là tổ...|    1|
|42527878|Liên Minh Công Lý...|    1|
|90282848|           5 đứa trẻ|    1|
|44179430|trực tiếp đá bóng...|    1|
|49569142|             DA BANH|    1|
|90592078|         on football|    2|
|96932783|    woori the virgin|    1|
|49195827|     u23 vief nam...|    1|
|41723086|trực tiếp việt na...|    2|
+--------+--------------------+-----+
only showing top 20 rows



In [9]:
data6 = data6.withColumnRenamed('count','TotalSearch')

In [10]:
data6.show()

+--------+--------------------+-----------+
| user_id|             keyword|TotalSearch|
+--------+--------------------+-----------+
| 8460879|Đao Kiếm Thần Vực...|          1|
| 3040638|             bóng đá|          1|
|98543977|    tái sinh báo thù|          1|
| 9030455|Trực tiếp U23 Việ...|          1|
|41789116|   tiên kiếm kỳ hiệp|          1|
|91999697|        bóng đá u23 |          1|
| 8221328|           kênh vtv6|          1|
| 0789087|    giao hữu quốc tế|          1|
| 0508207|không thèm yêu đư...|          3|
|42363805|             bóng đá|          1|
|49603005|truc tiep bong đá...|          1|
|42797857|hãy gọi tôi là tổ...|          1|
|42527878|Liên Minh Công Lý...|          1|
|90282848|           5 đứa trẻ|          1|
|44179430|trực tiếp đá bóng...|          1|
|49569142|             DA BANH|          1|
|90592078|         on football|          2|
|96932783|    woori the virgin|          1|
|49195827|     u23 vief nam...|          1|
|41723086|trực tiếp việt na...| 

In [11]:
data6 = data6.orderBy('user_id',ascending = False)

In [12]:
data6.show()

+--------+--------------------+-----------+
| user_id|             keyword|TotalSearch|
+--------+--------------------+-----------+
|99999941|             gintama|          1|
|99999941|       cẩm tú nam ca|          1|
|99999941|    thien menh hoang|          1|
|99999941|          thien mẻnh|          1|
|99999941|             vi uong|          1|
|99999941|        linh hon bac|          1|
|99999941|lấy danh nghĩa ng...|          1|
|99999793|               conan|          2|
|99999793|     vân tịch truyện|          4|
|99999793|      sở kiều truyện|          1|
|99999793|cuối cùng cũng đế...|          1|
|99999793|    như ý phương phi|          1|
|99999793|cô nàng trong trắ...|          1|
|99999793|       thám tử conan|          1|
|99999793| hải thượng phồn hoa|          1|
|99999793|     bằng chứng thép|          1|
|99999759|bí mật không thể nói|          1|
|99999759|cô nàng trong trắ...|          3|
|99999351|     phim thanh xuân|          1|
|99999351| GUI THOI THANH XUAN| 

In [14]:
# window = Window.partitionBy('user_id').orderBy(col('TotalSearch').desc())
# data = data.withColumn('Rank',row_number().over(window))

window = Window.partitionBy('user_id').orderBy(col('TotalSearch').desc())
data6 = data6.withColumn('Rank',row_number().over(window))

In [15]:
data6.show()

+--------+--------------------+-----------+----+
| user_id|             keyword|TotalSearch|Rank|
+--------+--------------------+-----------+----+
| 9724993|Tìm kiếm bằng giọ...|          1|   1|
|97249985|cuộc chiến giày g...|          2|   1|
|97249985|thử thách thần tượng|          2|   2|
|97249985|pháp y tần minh: ...|          1|   3|
|97249985|tại sao lại là oh...|          1|   4|
|97249985|           one piece|          1|   5|
|97249985|               conan|          1|   6|
|97249985|nụ hôn của giác q...|          1|   7|
|97249985|       running mùa 2|          1|   8|
|97249993|  người thầy y đức 2|          3|   1|
|97249993|    người thầy y đức|          1|   2|
|97250012|  trời đẹp em sẽ đến|          1|   1|
|97250033|    đời sống ngục tù|          3|   1|
|97250054|   chien long xa thu|          1|   1|
|97250057|                hoạ |          1|   1|
|97250057|    châu sinh như cố|          1|   2|
|97250067|        mộng hoa lục|          1|   1|
|97250067|          

In [16]:
data6 = data6.filter(col('Rank') == 1)

In [17]:
data6.show()

+--------+--------------------+-----------+----+
| user_id|             keyword|TotalSearch|Rank|
+--------+--------------------+-----------+----+
|97258098|nhân sinh nếu như...|          1|   1|
|97258102|cũng chỉ  là hạt bụi|          1|   1|
|97258106|            việt nam|          1|   1|
|97258157|           bác sĩ ma|          1|   1|
|97258178|      tranh thiên hạ|          2|   1|
|97258193|những chàng trai ...|          3|   1|
|97258249|            still 17|          1|   1|
|97258251|                 eve|          1|   1|
|97258309|          love logic|          1|   1|
|97258326|        mộng hoa lục|          1|   1|
|97258493|  thiên nga bóng đêm|          1|   1|
|97258523|   du sinh  xin chi |          2|   1|
|97258581|gạt đi Tình Thân ...|          1|   1|
|97258812|   pháp y tần minh 2|          1|   1|
|97258818|            HORIMIYA|          5|   1|
|97258824| thương ngày nắng về|          1|   1|
|97258877|     tokyo revengers|          1|   1|
| 9725900|yêu con gá

In [18]:
data6 = data6.withColumnRenamed('keyword','Most_Search')
data6 = data6.select('user_id','Most_Search')

In [19]:
data6.show()

+--------+--------------------+
| user_id|         Most_Search|
+--------+--------------------+
|97261297|  ngạ quỷ vùng tokyo|
|97261336|tôi thấy hoa vàng...|
| 9726134|            PLUNDERE|
|97261342|          DAU  PHA T|
|97261345|         running man|
|97261346|    đời sống ngục tù|
|97261433|            why her?|
|97261464|      phim hoạt hình|
|97261465|             tình cờ|
|97261540|       phim tình cảm|
| 9726159|chuyện tình bắc kinh|
|97261597|       we got maried|
|97261609|                NULL|
|97261616|                  uu|
|97261617|   nữ công tố viên 2|
|97261621| Định mệnh ánh trăng|
|97261720|     khu vườn bí mật|
|97261877|         running man|
|97261914|tháng 4 là lời nó...|
|97261923|  thiên nga bóng đêm|
+--------+--------------------+
only showing top 20 rows



In [4]:
def process_log_search(data):
    data = data.select('user_id','keyword')
    data = data.groupBy('user_id','keyword').count()
    data = data.withColumnRenamed('count','TotalSearch')
    data = data.orderBy('user_id',ascending = False )
    window = Window.partitionBy('user_id').orderBy(col('TotalSearch').desc())
    data = data.withColumn('Rank',row_number().over(window))
    data = data.filter(col('Rank') == 1)
    data = data.withColumnRenamed('keyword','Most_Search')
    data = data.select('user_id','Most_Search')
    return data 

In [5]:
data7 = process_log_search(data7)
data6 = process_log_search(data6)

In [22]:
data7.show()

+--------+--------------------+
| user_id|         Most_Search|
+--------+--------------------+
|97133188|      con tim sắt đá|
|97133258|kẻ trộm mặt trăng...|
|97133304|  bí mật nơi góc tối|
|97133340|học viện anh hùng...|
|97133411|    manh phi giá đáo|
|97133428|nỗi vương vấn của...|
|97133481|sáng như trăng tr...|
|97133487|              GHIBLI|
|97133517|   Bằng chứng thép 2|
|97133627|anh không thích t...|
|97133673|           Slam Dunk|
|97133687|    phép màu kì diệu|
| 9713370|           tây du ký|
|97133736|         date a live|
|97133759| khi hoa tình yêu nở|
|97133766|               jujut|
|97133842|         lao cuu mon|
|97133846|          luật sư kì|
|97133995|    ngon gio doi toi|
|97134003|     hana yori dango|
+--------+--------------------+
only showing top 20 rows



In [6]:
data6 = data6.withColumnRenamed('Most_Search','Most_Search_T6')
data7 = data7.withColumnRenamed('Most_Search','Most_Search_T7')    

In [13]:
#df_copy = df.select(*df.columns)
data6_cp = data6.select(*data6.columns)
data7_cp = data7.select(*data7.columns)

In [14]:
data67 = data6_cp.join(data7_cp,on="user_id",how="inner")

In [15]:
data67.show(2)

+-------+-----------------+--------------------+
|user_id|   Most_Search_T6|      Most_Search_T7|
+-------+-----------------+--------------------+
|0003361|      tây hành kỷ|         lộc đỉnh ký|
|0005748|lời nguyền ma lai|chuyen sinh thanh...|
+-------+-----------------+--------------------+
only showing top 2 rows



In [16]:
data67 = data67.limit(1000)
map6 = spark.read.csv(Mappath1,header=True)
data67 = data67.join(map6,on="Most_Search_T6",how="inner")
data67.show()

+--------------------+--------+--------------------+-----------+
|      Most_Search_T6| user_id|      Most_Search_T7|Category_T6|
+--------------------+--------+--------------------+-----------+
|   lời nguyền ma lai| 0005748|chuyen sinh thanh...| TV Series |
|    công chúa aurora| 0101498|   nam cung phu nhân| TV Series |
|  vương giả thiên hạ| 0150958|            dr stone| TV Series |
|yêu em từ cái nhì...| 0153643|hắc bạch vô song ...| TV Series |
|                 eve| 0176335|   nikita (season 2)| TV Series |
|       bắt ma phá án| 0218698|   khách sạn ma quái| TV Series |
|lấy danh nghĩa ng...| 0231670|         PACIFIC RIM| TV Series |
|        hoàng hậu ki| 0261295|   nguy cơ tiên sinh| TV Series |
|     minh lan truyện| 0262016|hoa nở đêm trăng thu| TV Series |
|     love in sadness| 0318156|nhất kiến khuynh tâm| TV Series |
|        mộng hoa lục| 0392220|          nữ luật sự| TV Series |
|        mộng hoa lục| 0392220|          nữ luật sự| TV Series |
|      prime provider| 04

In [20]:
map7 = spark.read.csv(Mappath2,header=True)
data67 = data67.join(map7,on="Most_Search_T7",how="inner")

In [21]:
data67.show()

+--------------------+-----------------+--------+-----------+-----------+
|      Most_Search_T7|   Most_Search_T6| user_id|Category_T6|Category_T7|
+--------------------+-----------------+--------+-----------+-----------+
|nhất kiến khuynh tâm|  love in sadness| 0318156| TV Series |  TV Series|
|      prime provider|   prime provider| 0417719| TV Series |  TV Series|
|                anna| vẻ đẹp đích thực| 0479873| TV Series |      Movie|
|      cá mực hầm mật|        đông cung| 0537443| TV Series |  TV Series|
|      con tim sắt đá|  love in sadness|06016004| TV Series |  TV Series|
|    hậu duệ mặt trời|         sao băng|06016278| TV Series |  TV Series|
|          fairy tail|        one piece|06028085|     Anime |      Anime|
|Thử Thách Thần Tượng|          ẩn danh|06028711| TV Series |    TV Show|
|          fairy tail|       fairy tail|06039284|     Anime |      Anime|
|    oán hận phải trả| thâm kế độc tình|06042523| TV Series |  TV Series|
|     phi vụ triệu đô|           gobli

In [25]:
data67 = data67.withColumn('CATEGORY_T6',trim(data67.Category_T6))

In [26]:
data67 = data67.withColumn('CATEGORY_T7',trim(data67.Category_T7))

In [27]:
data67.show()

+--------------------+-----------------+--------+-----------+-----------+
|      Most_Search_T7|   Most_Search_T6| user_id|CATEGORY_T6|CATEGORY_T7|
+--------------------+-----------------+--------+-----------+-----------+
|nhất kiến khuynh tâm|  love in sadness| 0318156|  TV Series|  TV Series|
|      prime provider|   prime provider| 0417719|  TV Series|  TV Series|
|                anna| vẻ đẹp đích thực| 0479873|  TV Series|      Movie|
|      cá mực hầm mật|        đông cung| 0537443|  TV Series|  TV Series|
|      con tim sắt đá|  love in sadness|06016004|  TV Series|  TV Series|
|    hậu duệ mặt trời|         sao băng|06016278|  TV Series|  TV Series|
|          fairy tail|        one piece|06028085|      Anime|      Anime|
|Thử Thách Thần Tượng|          ẩn danh|06028711|  TV Series|    TV Show|
|          fairy tail|       fairy tail|06039284|      Anime|      Anime|
|    oán hận phải trả| thâm kế độc tình|06042523|  TV Series|  TV Series|
|     phi vụ triệu đô|           gobli

In [28]:
data67 = data67.withColumn("Trending_Type",when(col("CATEGORY_T6") == col("CATEGORY_T7"),"Unchanged").otherwise("Changed"))

In [29]:
data67.show()

+--------------------+-----------------+--------+-----------+-----------+-------------+
|      Most_Search_T7|   Most_Search_T6| user_id|CATEGORY_T6|CATEGORY_T7|Trending_Type|
+--------------------+-----------------+--------+-----------+-----------+-------------+
|nhất kiến khuynh tâm|  love in sadness| 0318156|  TV Series|  TV Series|    Unchanged|
|      prime provider|   prime provider| 0417719|  TV Series|  TV Series|    Unchanged|
|                anna| vẻ đẹp đích thực| 0479873|  TV Series|      Movie|      Changed|
|      cá mực hầm mật|        đông cung| 0537443|  TV Series|  TV Series|    Unchanged|
|      con tim sắt đá|  love in sadness|06016004|  TV Series|  TV Series|    Unchanged|
|    hậu duệ mặt trời|         sao băng|06016278|  TV Series|  TV Series|    Unchanged|
|          fairy tail|        one piece|06028085|      Anime|      Anime|    Unchanged|
|Thử Thách Thần Tượng|          ẩn danh|06028711|  TV Series|    TV Show|      Changed|
|          fairy tail|       fai

In [30]:
data6_cp2 = data6.select(*data6.columns)
data7_cp2 = data7.select(*data7.columns)

In [33]:
data6_cp2 = data6_cp2.join(map6,on='Most_Search_T6',how='inner')

In [34]:
data7_cp2 = data7_cp2.join(map7,on='Most_Search_T7',how='inner')

In [35]:
data67_2 = data6_cp2.join(data7_cp2,on='user_id',how='inner')

In [37]:
data67_2.show()

+--------+--------------------+-----------+--------------------+-------------+
| user_id|      Most_Search_T6|Category_T6|      Most_Search_T7|  Category_T7|
+--------+--------------------+-----------+--------------------+-------------+
|97947758|    tam sinh tam thế| TV Series |định mệnh anh yêu em|    TV Series|
|98613952|               conan|     Anime |               conan|        Anime|
|98741227|      vượt ra tội ác| TV Series |          reply 1988|Sports Event |
|99542807|thanh gươm diệt q...|     Anime |              boruto|        Anime|
|99614186|    minh châu rực rỡ| TV Series |                anna|        Movie|
| 9991480|  thiên nga bóng đêm| TV Series |  thiên nga bóng đêm|    TV Series|
|99943742|            why her?| TV Series |          reply 1988|Sports Event |
|99197631|        nơi đảo xanh| TV Series |kẻ trộm mặt trăng...|        Movie|
|99214841|harry potter: cuộ...|     Movie |         running man|      TV Show|
|99840878|              bolero|Music Show |         

In [38]:
data67_2 = data67_2.withColumn('CATEGORY_T6',trim(data67_2.Category_T6))
data67_2 = data67_2.withColumn('CATEGORY_T7',trim(data67_2.Category_T7))

In [39]:
data67_2 = data67_2.withColumn("Trending_Type",when(col("CATEGORY_T6") == col("CATEGORY_T7"),"Unchanged").otherwise("Changed"))

In [40]:
data67_2 = data67_2.withColumn("Previous", when(col("Trending_Type") != "Unchanged", concat_ws(" to ", col("CATEGORY_T6"), col("CATEGORY_T7"))).otherwise("Unchanged"))

In [41]:
data67_2.show()

+--------+--------------------+-----------+--------------------+------------+-------------+--------------------+
| user_id|      Most_Search_T6|CATEGORY_T6|      Most_Search_T7| CATEGORY_T7|Trending_Type|            Previous|
+--------+--------------------+-----------+--------------------+------------+-------------+--------------------+
|97447399|  thiên nga bóng đêm|  TV Series|     phi vụ triệu đô|   TV Series|    Unchanged|           Unchanged|
|97947758|    tam sinh tam thế|  TV Series|định mệnh anh yêu em|   TV Series|    Unchanged|           Unchanged|
|98613952|               conan|      Anime|               conan|       Anime|    Unchanged|           Unchanged|
|98741227|      vượt ra tội ác|  TV Series|          reply 1988|Sports Event|      Changed|TV Series to Spor...|
|99542807|thanh gươm diệt q...|      Anime|              boruto|       Anime|    Unchanged|           Unchanged|
|99614186|    minh châu rực rỡ|  TV Series|                anna|       Movie|      Changed|  TV 